<a href="https://colab.research.google.com/github/2SEHI/Factory-Anomaly-Analysis/blob/main/1_Display_Sensor_Anomaly_Analysis_%EA%B2%B0%EC%B8%A1%EC%B9%98%EC%B2%98%EB%A6%AC%EC%99%80_%EC%83%81%EA%B4%80%EA%B3%84%EC%88%98%EC%A0%80%EC%9E%A5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 결측치 처리
- 결측치의 7개의 컬럼과 3개의 행 제거함.

## 컬럼 분리
- 결측치를 제거한 fact_data를 와 sensor, 폐기율으로 나눈 3가지를 파일에 각각 저장하였음.
- 전체 데이터 : fact_data
- sensor 데이터 : sensor_data
- 폐기율 관련 데이터 : trash_data

## 상관계수 파일 저장
- (결측치로 제거된)모든 sensor column끼리의 상관계수를 구하여 파일 저장
- all_col_corr



In [ ]:
# 공통 모듈 임포트
import numpy as np
import pandas as pd
import os
from scipy import stats
import seaborn as sns
# 깔끔한 그래프 출력을 위해
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)
mpl.rcParams['axes.unicode_minus'] = False

# 그림을 저장할 위치
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "Display_Sensor_Anomaly_Analysis"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("그림 저장:", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)
    
plt.rc('font', family='NanumBarunGothic') 

import warnings
warnings.filterwarnings(action='ignore')

np.random.seed(42)

In [ ]:
working_dir = '/content/drive/MyDrive/Colab Notebooks/data/Display_Sensor_Anomaly_Analysis/'

In [ ]:
fact_init_data = pd.read_csv(working_dir + 'factory_glass_2016.csv', encoding='cp949',index_col='dat...date.name.')

### 결측치 확인 
- 전부 Na인 컬럼이 7개 존재하며 1개의 결측치를 가진 컬럼들이 100개 이상 존재함.

In [ ]:
x = fact_init_data.isnull().sum().sort_values(ascending=False)
np.where(x > 0)

(array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
         13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
         26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
         39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
         52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
         65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
         78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
         91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
        104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
        117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
        130, 131, 132, 133]),)

- 전체 8145행 중에서 NaN가 8145행인 컬럼은 7개가 존재하므로 컬럼삭제

In [ ]:
print(fact_init_data.isnull().sum().sort_values(ascending=False).head(10))
fact_init_data = fact_init_data.dropna(how='all', axis=1)

ROOF.HEATING.Zone52.1JIC31152.PV        8145
UPPER.RET..Z218.TMP.1TIC42601.PV        8145
ROOF.HEATING.Zone78.1JIC31178.PV        8145
ROOF.HEATING.Zone76.1JIC31176.PV        8145
LEHR.DRV.MTR..M19.2.CUR.1WII40135.PV    8145
DCS_Offset_RPM_M14.1WOS45114_PV         8145
DCS_Offset_RPM_M13.1WOS45113_PV         8145
DB.HEAT.TOP.4.1.2.TMP.CT.1JI40304.PV       1
DB.HEAT.TOP.5.1.2.TMP.CT.1JI40305.PV       1
DB.HEAT.TOP.7.1.2.TMP.CT.1JI40307.PV       1
dtype: int64


In [ ]:
fact_init_data.isnull().sum().sort_values(ascending=False).head(100)

DB.HEAT.BOTTOM.1.1JIC40308.PV           1
DB.HEAT.TOP.4.1.2.TMP.CT.1JI40304.PV    1
DROSS.BOX.N2.BTM.HT..L..1JI39009.PV     1
DROSS.BOX.N2.BTM.HT..R..1JI39010.PV     1
DROSSBOX.N2.SCN.L.1JI39011_PV           1
                                       ..
X.1.LEHR.O2.ANALYZER.1AI44111.PV        1
X.2.LEHR.SO2.ANALYZER.1AI44112.PV       1
X.2.LEHR.O2.ANALYZER.1AI44113.PV        1
S_C1_C2_L_R                             1
CASHING.COLD.AIR.TMP.1TIC33603.PV       1
Length: 100, dtype: int64

## 결측치가 존재하는 행을 평균으로 결측치 대체

In [ ]:
fact_data_filled = fact_init_data.fillna(fact_init_data.mean())

## 결측치가 존재하는 행 출력과 삭제
- 3행에 모두 몰려있으므로 행을 삭제한다.

In [ ]:
print(fact_init_data[fact_init_data.isnull().any(axis=1)])
drop_index = list(fact_init_data[fact_init_data.isnull().any(axis=1)].index)
print(drop_index)

# 각 행의 NaN개수
print(fact_init_data.iloc[3754].isnull().sum())
print(fact_init_data.iloc[6310].isnull().sum())
print(fact_init_data.iloc[8044].isnull().sum())

# NaN가 있는 행 삭제
fact_data = fact_init_data.drop(index=drop_index, axis=0)

                  LIFT.OUT.ROL.MTR.M1.SPD.1WSI40101  ...  R.Vac
dat...date.name.                                     ...       
2016-06-24 04                              1.467846  ...    0.0
2016-10-13 09                             -0.846817  ...    0.0
2016-12-27 17                             -0.846817  ...    0.0

[3 rows x 833 columns]
['2016-06-24 04', '2016-10-13 09', '2016-12-27 17']
1
39
87


## 컬럼 분리
- 결측치를 제거한 fact_data를 와 sensor, 폐기율으로 나눈 3가지를 파일에 각각 저장하였음.
- 전체 데이터 : fact_data
- sensor 데이터 : sensor_data
- 폐기율 관련 데이터 : trash_data

In [ ]:
# 폐기율 컬럼
trash_columns = ['투입.LEFT.','투입.RIGHT.','ROLLDOWN.폐기.LEFT.','진공패드.폐기.LEFT.','ROLLDOWN.폐기.RIGHT.','진공패드.폐기.RIGHT.','폐기율...','ROLLDOWN.진공패드.폐기율..LEFT.','ROLLDOWN.진공패드.폐기율..RIGHT.','ROLLDOWN.진공패드.폐기율..전체.','L.RD','L.Vac','R.RD','R.Vac',]
sensor_data = fact_data.drop(columns=trash_columns)
trash_data = fact_data[trash_columns]

fact_data.to_csv(working_dir + 'fact_data.csv')
sensor_data.to_csv(working_dir + 'sensor_data.csv')
trash_data.to_csv(working_dir + 'trash_data.csv')

## 모든 sensor 컬럼에 대한 상관관계를 파일로 저장
- ❗❗총 10분 소요
- 파일명 
```
corr_df0-100.csv
corr_df100-200.csv
corr_df200-300.csv
corr_df300-500.csv
corr_df500-700.csv
corr_df700-819.csv
```


In [ ]:
# 모든 sensor 컬럼에 대한 상관계수를 저장하는 함수
def save_coef(start_num, end_num):
    # 컬럼1, 컬럼2, 상관계수를 담는 list
    corr_tp = []

    # 두 컬럼의 상관계수를 구하는 for문
    for i in range(start_num, end_num):
        for j in range(i+1, len(sensor_columns)):
            # 상관계수 저장
            c = sensor_data[[sensor_columns[i],sensor_columns[j]]].corr().abs()
            # DataFrame을 Series로 변환
            s = c.unstack()
            # 정렬(가장 높은 상관계수는 자기 자신이므로 3또는 4번째가 두 컬럼의 상관계수)
            so = s.sort_values(kind="quicksort", ascending=False)
            # (컬럼1, 컬럼2, 상관계수)를 list에 저장
            # 가장 높은 상관계수는 자기 자신이므로 3또는 4번째가 두 컬럼의 상관계수
            corr_tp.append((sensor_columns[i],sensor_columns[j],so[3]))
    corr_df = pd.DataFrame(corr_tp, columns=['col1','col2','coef'])
    file_name = 'corr_df' + str(start_num) +'-'+ str(end_num)

    corr_df.to_csv(working_dir + file_name + '.csv')


In [ ]:
sensor_columns = sensor_data.columns

#*************************************
# s : 시작 컬럼 index
# e : 마지막 컬럼 index
#*************************************
index_list = [(0,100),(100, 200), (200, 300), (300,500),(500, 700), (700,len(sensor_columns))]
for s, e in index_list:
    save_coef(s,e)

In [ ]:
def get_coef(start_num, end_num):
    file_name = 'corr_df' + str(start_num) +'-'+ str(end_num)
    coef_df  = pd.read_csv(working_dir + file_name + '.csv', usecols=(1,2,3))
    coef_df = coef_df.sort_values(by='coef', ascending=False)
    return coef_df
   
coef_list_1 = get_coef(0, 100)
coef_list_2 = get_coef(100, 200)
coef_list_3 = get_coef(200, 300)
coef_list_4 = get_coef(300, 500)
coef_list_5 = get_coef(500, 700)
coef_list_6 = get_coef(700, len(sensor_columns))

In [ ]:
# 추출할 상관계수 크기 
compare_num = 1.0

high_coef_df = pd.DataFrame(columns=['col1','col2','coef'])
def get_high_coef(coef_df, compare_num):
    coef_df = coef_df[coef_df['coef'] >= compare_num]
    global high_coef_df
    high_coef_df = high_coef_df.append(coef_df,ignore_index=True)

get_high_coef(coef_list_1,compare_num)
get_high_coef(coef_list_2,compare_num)
get_high_coef(coef_list_3,compare_num)
get_high_coef(coef_list_4,compare_num)
get_high_coef(coef_list_5,compare_num)
get_high_coef(coef_list_6,compare_num)

file_name = 'high_coef_df_upper_' + str(compare_num)
# csv 파일로 저장
high_coef_df.to_csv(working_dir + file_name + '.csv')

In [ ]:
# csv 파일불러오기
high_coef_df = pd.read_csv(working_dir + file_name + '.csv',usecols=(1,2,3))
high_coef_df

,col1,col2,coef
0,LEHR.DRV.MTR..M6.1.SPD.1WSI40108,DRIVE.MOTOR.M19.1.SPEED.1WSI40134,1.0
1,DAMP.POSITION.4.BAY.CTRL.1TIC33424.PV,DAMP.POSITION.6.BAY.CTRL.1TIC33426.PV,1.0
2,ATM.FLOW.SECT.6.N2.H2.1FI33410.PV,ATM.FLOW.SECT.9.N2.H2.1FIC33416.PV,1.0
3,LEHR.DRV.MTR..M7.1.SPD.1WSI40110,LEHR.DRV.MTR..M14.1.SPD.1WSI40124,1.0
4,ATM.FLOW.SECT.6.N2.H2.1FI33410.PV,ATM.FLOW.SECT.10.N2.H2.1FIC33418.PV,1.0
...,...,...,...
97,LEHR.DRV.MTR..M11.2.CUR.1WII40119.PV,LEHR.DRV.MTR..M12.2.CUR.1WII40121.PV,1.0
98,LEHR.DRV.MTR..M10.1.CUR.1WII40116.PV,TMP.TIN..BAY.4.LEFT.1TI30203.PV,1.0
99,LEHR.DRV.MTR..M9.1.CUR.1WII40114.PV,LEHR.DRV.MTR..M10.1.CUR.1WII40116.PV,1.0
100,LEHR.DRV.MTR..M10.1.CUR.1WII40116.PV,LEHR.DRV.MTR..M17.2.CUR.1WII40131.PV,1.0
